In [1]:
!pip install -U scikit-learn

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import PowerTransformer, TargetEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.pipeline import Pipeline  # Importing from imblearn
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_val_score

In [3]:
# Load data
train = pd.read_csv('/kaggle/input/ml-fundamentals-and-applications-2024-10-01/final_proj_data.csv')
valid = pd.read_csv('/kaggle/input/ml-fundamentals-and-applications-2024-10-01/final_proj_test.csv')

In [4]:
train.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,Var11,Var12,Var13,Var14,Var15,Var16,Var17,Var18,Var19,Var20,Var21,Var22,Var23,Var24,Var25,Var26,Var27,Var28,Var29,Var30,Var31,Var32,Var33,Var34,Var35,Var36,Var37,Var38,Var39,Var40,...,Var192,Var193,Var194,Var195,Var196,Var197,Var198,Var199,Var200,Var201,Var202,Var203,Var204,Var205,Var206,Var207,Var208,Var209,Var210,Var211,Var212,Var213,Var214,Var215,Var216,Var217,Var218,Var219,Var220,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230,y
0,NaN,NaN,NaN,NaN,NaN,812.0,14.0,NaN,NaN,NaN,NaN,NaN,1252.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,156.0,195.0,NaN,0.0,40.0,NaN,NaN,286.96,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,4850466.0,NaN,NaN,...,KXMrEyXXnK,g62hiBSaKg,SEuy,taul,1K8T,IvdZ,fhk21Ss,Pkku4gO,pbHkKHa,smXZ,4MV4,9_Y1,SkZj,VpdQ,itlM,7M47J5GA0pTYIFxg5uy,kIsH,NaN,uKAI,L84s,WsRVNrF85oPU_,NaN,E4QF4eX,NaN,11pRpOD,G8WR,cJvF,AU8_WTd,4UxGlow,zCkv,catzS2D,jySVZNlOJy,NaN,xG3x,Aoh3,ZI9m,ib5G6X1eUxUn6,mj86,NaN,0
1,NaN,NaN,NaN,NaN,NaN,2688.0,7.0,NaN,NaN,NaN,NaN,NaN,8820.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,364.0,455.0,NaN,4.0,288.0,NaN,NaN,200.00,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,132072.0,NaN,NaN,...,8Knvyx875g,2Knk1KF,SEuy,taul,1K8T,lK27,eQgnKxT,Hz673939hbJdw,ELe4G18,smXZ,sbhw,9_Y1,MBhA,VpdQ,lVqb,me75fM6ugJ,kIsH,NaN,uKAI,L84s,XfqtO3UdzaXh_,NaN,kfKvbJ9,NaN,11pEY2j,5smi,UYBR,AU8pNoi,GpvRJ5l,oslk,i06ocsg,LM8l689qOp,NaN,kG3k,WqMG,RAYp,55YFVY9,mj86,NaN,0
2,NaN,NaN,NaN,NaN,NaN,1015.0,14.0,NaN,NaN,NaN,NaN,NaN,1784.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,136.0,170.0,NaN,2.0,40.0,NaN,NaN,294.48,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,3223524.0,NaN,NaN,...,MfKrEyQtC3,RO12,SEuy,taul,1K8T,487l,iJzviRg,76j2P_OLn0,pOtdZA9,smXZ,7KBn,9_Y1,SkZj,VpdQ,hAFG,7M47J5GA0pTYIFxg5uy,kIsH,NaN,g5HH,L84s,cOJGGr4F_wga1,NaN,3_oHWGU,NaN,kq0mb4H,8m7I,cJvF,FzaX,ch2oGfM,zCkv,P6pu4Vl,LM8l689qOp,NaN,kG3k,Aoh3,ZI9m,R4y5gQQWY8OodqDV,am7c,NaN,0
3,NaN,NaN,NaN,NaN,NaN,168.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0,30.0,NaN,0.0,0.0,NaN,NaN,644.24,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,2135430.0,NaN,NaN,...,Qu0qrQKzJV,RO12,NaN,taul,1K8T,487l,3cunsla,ZIX6Y9cVy5,NaN,NaN,6lcF,9_Y1,15m3,09_Q,zm5i,me75fM6ugJ,kIsH,NaN,uKAI,Mtgm,NhsEn4L,NaN,NaN,NaN,7Wwz9n7,d4ij,cJvF,FzaX,kH5mFX7,oslk,BNrD3Yd,LM8l689qOp,NaN,NaN,FSa2,RAYp,F2FyR07IdsN7I,NaN,NaN,0
4,NaN,NaN,NaN,NaN,NaN,14.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.0,45.0,NaN,0.0,0.0,NaN,NaN,239.84,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,3110400.0,NaN,NaN,...,DHerjIKQ76,RO12,NaN,taul,1K8T,YPhp,65ylkJq,glRbITbnHQ,NaN,NaN,UCEj,9_Y1,7WNq,VpdQ,IYzP,me75fM6ugJ,kIsH,NaN,uKAI,L84s,NhsEn4L,NaN,NaN,NaN,kZJtVhC,NEOV,cJvF,FzaX,x_lYlW4,oslk,3B1QowC,LM8l689qOp,NaN,NaN,WqMG,RAYp,F2FyR07IdsN7I,NaN,NaN,0


In [5]:
# Define target variable
target = train.pop('y')

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 230 entries, Var1 to Var230
dtypes: float64(191), int64(1), object(38)
memory usage: 17.5+ MB


In [7]:
# Viewing the number of missing values 
# (as we can see there are quite a lot of them in some features)
train.isna().sum()

Var1       9867
Var2       9734
Var3       9734
Var4       9720
Var5       9759
          ...  
Var226        0
Var227        0
Var228        0
Var229     5561
Var230    10000
Length: 230, dtype: int64

In [8]:
# Remove columns with more than 30% missing values or with more than 30% unique in categorical
threshold = 0.3 * len(train)
columns_to_drop = train.columns[train.isnull().sum() > threshold]

train.drop(columns=columns_to_drop, inplace=True)
valid.drop(columns=columns_to_drop, inplace=True)

In [9]:
# Define numeric and categorical features
train_num = train.select_dtypes(include=np.number).columns
train_cat = train.select_dtypes(include='object').columns

In [10]:
# Viewing the distribution of features 
# (as we see the distribution of features, so they should be balanced)
train[train_num].skew()

Var6      12.262550
Var7       1.352816
Var13     10.168912
Var21     16.478526
Var22     16.536072
Var24      8.071358
Var25     13.496533
Var28      4.731732
Var35      6.444871
Var38      1.107181
Var44     17.816443
Var57     -0.014228
Var65      2.085809
Var73      0.841976
Var74     18.050388
Var76      2.802532
Var78      5.808068
Var81      2.921827
Var83     26.155132
Var85     13.177699
Var109    21.595046
Var112    12.883710
Var113    -1.074561
Var119    17.604608
Var123    25.332244
Var125    25.090343
Var126     0.635994
Var132     4.273061
Var133     1.290760
Var134     2.615101
Var140     5.462813
Var143    12.838947
Var144     1.213175
Var149    11.391872
Var153    -0.282752
Var160    14.986289
Var163     4.729003
Var173    20.245126
Var181     5.418719
dtype: float64

In [11]:
# View the number of unique values for each feature
target.value_counts()

y
0    8695
1    1305
Name: count, dtype: int64

In [12]:
# Numeric pipeline
numeric_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer()),  # Imputation
    ('scaler', PowerTransformer())  # Scaling
])

# Categorical pipeline
categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Imputation
    ('encoder', TargetEncoder(random_state=42))  # Encoding
])

# Combine pipelines in a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline, train_num),
        ('cat', categorical_pipeline, train_cat)
    ])

# Define the model pipeline with SMOTE
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=42)),  # SMOTE for resampling
    ('classifier', GradientBoostingClassifier(random_state=42))  # Classifier
])

In [13]:
# Split data into train and test
X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.2, random_state=42)

In [14]:
# Fit the model pipeline
model_pipeline.fit(X_train, y_train)

# Perform cross-validation
cv_scores = cross_val_score(model_pipeline, X_train, y_train, cv=5, scoring='balanced_accuracy')

# Calculate mean and standard deviation of cross-validation scores
mean_cv_score = cv_scores.mean()
std_cv_score = cv_scores.std()

print(f'Cross-validated balanced accuracy: {mean_cv_score:.1%} ± {std_cv_score:.1%}')

Cross-validated balanced accuracy: 86.6% ± 1.1%


In [15]:
# Final model training with specific hyperparameters
hyperparams = {
    'learning_rate': 0.1,
    'max_depth': 5,
    'n_estimators': 100
}

# Create a new pipeline with the final hyperparameters
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=42)),
    ('classifier', GradientBoostingClassifier(**hyperparams, random_state=42))  # Using final hyperparams
])

# Fit the final model
model.fit(train, target)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   PowerTransformer())]),
                                                  Index(['Var6', 'Var7', 'Var13', 'Var21', 'Var22', 'Var24', 'Var25', 'Var28',
       'Var35', 'Var38', 'Var44', 'Var57', 'Var65', 'Var73', 'Var74', 'Var76',
       'Var78', 'Var81', 'Var83', 'Var85', 'Var109', 'Var112', 'Var113',
       'Var119', 'V...
                                                  Index(['Var192', 'Var193', 'Var195', 'Var196', 'Var197', 'Var198', 'Var199',
       'Var202', 'Var203', 'Var204', 'Var205', 'Var206', 'Var207', 'Var208',
       'Var210', 'Var211', 'Var212', 'Var216', 'Var217', 'Var218', 'Var219',
       'Var220', 'Var221', 'Var222', 'Var223', 'Var226', 'Var227', 'Var228'],
      dtype='object'))])),
                ('smote', SMOTE(random_state=42)),
                ('classifier',
                 GradientBoostingClassifier(max_depth=5, random_state=42))])

In [16]:
# Predict on validation data
valid_preds = model.predict(valid)
output = pd.DataFrame({'index': valid.index, 'y': valid_preds})
output.to_csv('submission.csv', index=False)